In [1]:
import tensorflow as tf
import numpy as np
from collections import Counter
import pandas

In [2]:
vocab = Counter()

text = "Hi from Brazil"

for word in text.split(' '):
    word_lowercase = word.lower()
    vocab[word_lowercase]+=1
        
def get_word_2_index(vocab):
    word2index = {}
    for i,word in enumerate(vocab):
        word2index[word] = i
        
    return word2index

In [3]:
word2index = get_word_2_index(vocab)

total_words = len(vocab)
matrix = np.zeros((total_words),dtype=float)

for word in text.split():
    matrix[word2index[word.lower()]] += 1
    
print("Hi from Brazil:", matrix)

Hi from Brazil: [ 1.  1.  1.]


In [4]:
matrix = np.zeros((total_words),dtype=float)
text = "Hi"
for word in text.split():
    matrix[word2index[word.lower()]] += 1
    
print("Hi:", matrix)

Hi: [ 0.  0.  1.]


In [5]:
from sklearn.datasets import fetch_20newsgroups
categories = ["comp.graphics", "sci.space", "rec.sport.baseball"]

newsgroups_train = fetch_20newsgroups(subset='train', categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories)

In [6]:
print('total texts in train:',len(newsgroups_train.data))
print('total texts in test:',len(newsgroups_test.data))

total texts in train: 1774
total texts in test: 1180


In [7]:
vocab = Counter()

for text in newsgroups_train.data:
    for word in text.split(' '):
        vocab[word.lower()]+=1
        
for text in newsgroups_test.data:
    for word in text.split(' '):
        vocab[word.lower()]+=1

In [8]:
print("Total words:",len(vocab))

Total words: 119930


In [9]:
total_words = len(vocab)

def get_word_2_index(vocab):
    word2index = {}
    for i,word in enumerate(vocab):
        word2index[word.lower()] = i
        
    return word2index

word2index = get_word_2_index(vocab)

print("Index of the word 'the':",word2index['the'])

Index of the word 'the': 96081


In [10]:
def get_batch(df,i,batch_size):
    batches = []
    results = []
    texts = df.data[i*batch_size:i*batch_size+batch_size]
    categories = df.target[i*batch_size:i*batch_size+batch_size]
    for text in texts:
        layer = np.zeros(total_words,dtype=float)
        for word in text.split(' '):
            layer[word2index[word.lower()]] += 1
            
        batches.append(layer)
        
    for category in categories:
        y = np.zeros((3),dtype=float)
        if category == 0:
            y[0] = 1.
        elif category == 1:
            y[1] = 1.
        else:
            y[2] = 1.
        results.append(y)
            
     
    return np.array(batches),np.array(results)

In [11]:
# Parameters
learning_rate = 0.1
training_epochs = 20
batch_size = 150
display_step = 1

# Network Parameters
n_hidden_1 = 100      # 1st layer number of features
n_hidden_2 = 100       # 2nd layer number of features
n_hidden_3 = 100   #experimental 3rd layer
n_input = total_words # Words in vocab
n_classes = 3         # Categories: graphics, sci.space and baseball

input_tensor = tf.placeholder(tf.float32,[None, n_input],name="input")
output_tensor = tf.placeholder(tf.float32,[None, n_classes],name="output") 

In [12]:
def multilayer_perceptron(input_tensor, weights, biases):
    L1_mult = tf.matmul(input_tensor, weights['h1'])
    L1_add  = tf.add(L1_mult, biases['b1'])
    L1_acti = tf.nn.relu(L1_add)
    
    #hidden layer w/ RELU activation
    L2_mult = tf.matmul(L1_acti, weights['h2'])
    L2_add  = tf.add(L2_mult, biases['b2'])
    L2_acti = tf.nn.relu(L2_add)
    
    
    #Output layer w/ linear activation
    Out_mult = tf.matmul(L2_acti, weights['out'])
    Out_add  = Out_mult + biases['out']
    
    return Out_add

In [13]:
def three_layer_perceptron(input_tensor, weights, biases):
    L1_mult = tf.matmul(input_tensor, weights['h1'])
    L1_add  = tf.add(L1_mult, biases['b1'])
    L1_acti = tf.nn.relu(L1_add)
    
    #hidden layer w/ RELU activation
    L2_mult = tf.matmul(L1_acti, weights['h2'])
    L2_add  = tf.add(L2_mult, biases['b2'])
    L2_acti = tf.nn.relu(L2_add)
    
    #hidden layer w/ RELU activation
    L3_mult = tf.matmul(L2_acti, weights['h3'])
    L3_add  = tf.add(L3_mult, biases['b3'])
    L3_acti = tf.nn.relu(L3_add)
    
    
    #Output layer w/ linear activation
    Out_mult = tf.matmul(L3_acti, weights['out'])
    Out_add  = Out_mult + biases['out']
    
    return Out_add

In [14]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3])),
    'out': tf.Variable(tf.random_normal([n_hidden_3, n_classes]))
    #'out2': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'b3': tf.Variable(tf.random_normal([n_hidden_3])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
#prediction = multilayer_perceptron(input_tensor, weights, biases)
prediction = three_layer_perceptron(input_tensor, weights, biases)

# Define loss and optimizer
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=output_tensor))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

# Initializing the variables
init = tf.global_variables_initializer()

In [15]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    
    # Training cycle
    for epoch in range(training_epochs):

        
        avg_cost = 0.
        total_batch = int(len(newsgroups_train.data)/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x,batch_y = get_batch(newsgroups_train,i,batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            c,_ = sess.run([loss,optimizer], feed_dict={input_tensor: batch_x,output_tensor:batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        #for variable learning rate
        if avg_cost < 100:
            learning_rate= .001
            optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
        elif avg_cost < 1000:
            learning_rate= .01
            optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "loss=", \
                "{:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(output_tensor, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    total_test_data = len(newsgroups_test.target)
    batch_x_test,batch_y_test = get_batch(newsgroups_test,0,total_test_data)
    print("Accuracy:", accuracy.eval({input_tensor: batch_x_test, output_tensor: batch_y_test}))

Epoch: 0001 loss= 15507.386524547
Epoch: 0002 loss= 1378.256100741
Epoch: 0003 loss= 338.229623101


FailedPreconditionError: Attempting to use uninitialized value beta1_power_1
	 [[Node: beta1_power_1/read = Identity[T=DT_FLOAT, _class=["loc:@Variable"], _device="/job:localhost/replica:0/task:0/cpu:0"](beta1_power_1)]]

Caused by op 'beta1_power_1/read', defined at:
  File "/Users/josephhowarth/anaconda3/envs/tensorflow/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/josephhowarth/anaconda3/envs/tensorflow/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/josephhowarth/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/josephhowarth/anaconda3/envs/tensorflow/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/josephhowarth/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoo

2nd trial
#Parameters
learning_rate = 0.01
training_epochs = 10 t
batch_size = 150
display_step = 1

#Network Parameters
n_hidden_1 = 10      # 1st layer number of features
n_hidden_2 = 10       # 2nd layer number of features
n_input = total_words # Words in vocab
n_classes = 3         # Categories: graphics, sci.space and baseball

input_tensor = tf.placeholder(tf.float32,[None, n_input],name="input")
output_tensor = tf.placeholder(tf.float32,[None, n_classes],name="output") 

Epoch: 0001 loss= 135.036456368
Epoch: 0002 loss= 28.945528377
Epoch: 0003 loss= 16.062389807
Epoch: 0004 loss= 7.733895800
Epoch: 0005 loss= 4.695653872
Epoch: 0006 loss= 3.015233267
Epoch: 0007 loss= 2.054380758
Epoch: 0008 loss= 1.708085900
Epoch: 0009 loss= 1.342620275
Epoch: 0010 loss= 1.057100686
Optimization Finished!
Accuracy: 0.499153

3rd trial

#Parameters
same ^^

#Network Parameters
same ^^ expect:
n_hidden_3 = 10   #experimental 3rd layer


input_tensor = tf.placeholder(tf.float32,[None, n_input],name="input")
output_tensor = tf.placeholder(tf.float32,[None, n_classes],name="output") 
Epoch: 0001 loss= 305.216054049
Epoch: 0002 loss= 64.048841303
Epoch: 0003 loss= 41.561178728
Epoch: 0004 loss= 17.813625336
Epoch: 0005 loss= 9.880973643
Epoch: 0006 loss= 6.213255882
Epoch: 0007 loss= 4.466571808
Epoch: 0008 loss= 3.334279613
Epoch: 0009 loss= 2.121624025
Epoch: 0010 loss= 1.598570704
Optimization Finished!
Accuracy: 0.435593

4th trial 
#Parameters
learning_rate = 0.1

#Network Parameters
same ^^ expect:
n_hidden_3 = 10   #experimental 3rd layer

Epoch: 0001 loss= 64.306565350
Epoch: 0002 loss= 1.613156026
Epoch: 0003 loss= 1.241018111
Epoch: 0004 loss= 1.115476229
Epoch: 0005 loss= 1.090250947
Epoch: 0006 loss= 1.078835011
Epoch: 0007 loss= 1.075026642
Epoch: 0008 loss= 1.069630363
Epoch: 0009 loss= 1.067430366
Epoch: 0010 loss= 1.065403862
Optimization Finished!
Accuracy: 0.388136

5th trial

n_hidden_1 = 30      # 1st layer number of features
n_hidden_2 = 30       # 2nd layer number of features
n_hidden_3 = 30   #experimental 3rd layer

5th trial

#Parameters
learning_rate = 0.1
training_epochs = 10
batch_size = 150
display_step = 1

#Network Parameters
n_hidden_1 = 30      # 1st layer number of features
n_hidden_2 = 30       # 2nd layer number of features
n_hidden_3 = 30   #experimental 3rd layer
n_input = total_words # Words in vocab
n_classes = 3         # Categories: graphics, sci.space and baseball

Epoch: 0001 loss= 1701.295199308
Epoch: 0002 loss= 284.312942505
Epoch: 0003 loss= 98.929033800
Epoch: 0004 loss= 33.875180071
Epoch: 0005 loss= 10.407883243
Epoch: 0006 loss= 2.965831166
Epoch: 0007 loss= 1.559648185
Epoch: 0008 loss= 0.884519023
Epoch: 0009 loss= 0.506064798
Epoch: 0010 loss= 0.347801045
Optimization Finished!
Accuracy: 0.798305

6th trial
learning_rate = 0.01
training_epochs = 20
batch_size = 150
display_step = 1

#Network Parameters
n_hidden_1 = 100      # 1st layer number of features
n_hidden_2 = 100       # 2nd layer number of features
n_hidden_3 = 100   #experimental 3rd layer
n_input = total_words # Words in vocab
n_classes = 3         # Categories: graphics, sci.space and baseball

Epoch: 0001 loss= 14807.826704545
Epoch: 0002 loss= 2737.399480646
Epoch: 0003 loss= 751.383565729
Epoch: 0004 loss= 835.108093782
Epoch: 0005 loss= 70.875464237
Epoch: 0006 loss= 61.981453153
Epoch: 0007 loss= 15.870573872
Epoch: 0008 loss= 1.877646958
Epoch: 0009 loss= 0.000000000
Epoch: 0010 loss= 0.000002227
Epoch: 0011 loss= 0.315085238
Epoch: 0012 loss= 148.914579928
Epoch: 0013 loss= 48.858623335
Epoch: 0014 loss= 172.233125036
Epoch: 0015 loss= 15.083169409
Epoch: 0016 loss= 4.523800590
Epoch: 0017 loss= 19.869936380
Epoch: 0018 loss= 125.841578397
Epoch: 0019 loss= 58.460036191
Epoch: 0020 loss= 61.418470534
Optimization Finished!
Accuracy: 0.640678


ADDED VARIABLE LEARNING_RATE
````````````````````````````